# 15. 심층 합성곱 신경망으로 이미지 분류

구성: 여러개의 `합성곱 층`, 학습되는 파라미터가 없는 `풀링 층` (서브 샘플링 층), 하나 이상의 완전 연결 층 - `다층 퍼셉트론 층`  

## 15.1 합성곱 신경망의 구성 요소

### 15.1.2 이산 합성곱 수행

In [1]:
# 1차원 합성곱 구현
import numpy as np
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int((len(x_padded)-len(w_rot))/s+1), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

In [2]:
# 2차원 합성곱 구현
import scipy.signal

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0], p[1]:p[1]+X_orig.shape[1]] = X_orig
    
    res = []
    for i in range(0, int((X_padded.shape[0]-W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1]-W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

## 15.2 기본 구성 요소를 사용하여 심층 합성곱 신경망 구성

### 15.2.2 드롭아웃으로 신경망 규제

In [3]:
# 일단 L2 regularization
from tensorflow import keras
conv_layer = keras.layers.Conv2D(
    filters=16,
    kernel_size=(3, 3),
    kernel_regularizer=keras.regularizers.l2(0.001))

In [4]:
fc_layer = keras.layers.Dense(
    units=16,
    kernel_regularizer=keras.regularizers.l2(0.001))

### 15.2.3 분류를 위한 손실 함수

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
# 이진 크로스 엔트로피 - 이진 분류를 위한 손실함수
bce_probas = tf.keras.losses.BinaryCrossentropy(from_logits=False)  # 확률로 계산 결과가 나옴
bce_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)

logits = tf.constant([0.8])
probas = tf.keras.activations.sigmoid(logits)
tf.print(
    'BCE (확률): {:.4f}'.format(
        bce_probas(y_true=[1], y_pred=probas)),
    '(로짓): {:.4f}'.format(
        bce_logits(y_true=[1], y_pred=logits)))

2022-04-26 23:02:09.741466: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
BCE (확률): 0.3711 (로짓): 0.3711


In [13]:
# 범주형 크로스 엔트로피
cce_probas = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
cce_logits = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

logits = tf.constant([[1.5, 0.8, 2.1]])
probas = tf.keras.activations.softmax(logits)
tf.print(
    'CCE (확률): {:.4f}'.format(cce_probas(y_true=[[0, 0, 1]], y_pred=probas)),
    '(로짓): {:.4f}'.format(cce_logits(y_true=[[0, 0, 1]], y_pred=logits)))

CCE (확률): 0.5996 (로짓): 0.5996


In [14]:
# 희소 범주형 크로스 엔트로피
sp_cce_probas = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
sp_cce_logits = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

tf.print('Sparse CCE (확률): {:.4f}'.format(sp_cce_probas(y_true=[2], y_pred=probas)),
         '(로짓): {:.4f}'.format(sp_cce_logits(y_true=[2], y_pred=logits)))

Sparse CCE (확률): 0.5996 (로짓): 0.5996


## 15.3 텐서플로를 사용하여 심층 합성곱 신경망 구현

### 15.3.2 데이터 적재와 전처리

In [1]:
import tensorflow_datasets as tfds

mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
datasets = mnist_bldr.as_dataset(shuffle_files=False)
mnist_train_orig = datasets['train']
mnist_test_orig = datasets['test']

2022-04-27 06:55:10.805904: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# validation set 나누기
import tensorflow as tf

BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20
mnist_train = mnist_train_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32) / 255.0,
                  tf.cast(item['label'], tf.int32))
)
mnist_test = mnist_test_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32) / 255.0,
                  tf.cast(item['label'], tf.int32))
)
tf.random.set_seed(1)
mnist_train = mnist_train.shuffle(buffer_size=BUFFER_SIZE,
                                  reshuffle_each_iteration=False)
mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE)
mnist_test = mnist_train.skip(10000).batch(BATCH_SIZE)